In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os

#file path
fp = r"C:\Users\renhu\OneDrive\Desktop\205.2 DATA\rawdata\Average_Daily_Traffic_Counts.geojson"

#read file
traffic = gpd.read_file(fp)
print("origional shape:", traffic.shape)
print("Columns:", list(traffic.columns))

#keep columns 
keep_cols = [ "road_id", "road_name", "start_name", "end_name",
    "location", "latest", "count_date", "peak_hour",
    "adt", "peaktraffic",
    "pccar", "pclcv", "pcmcv", "pchcvi", "pchcvii", "pcbus", "pcheavy",
    "geometry"
]
traffic = traffic[[col for col in keep_cols if col in traffic.columns]].copy()


origional shape: (13093, 23)
Columns: ['OBJECTID', 'carr_way_no', 'road_id', 'road_name', 'start_name', 'end_name', 'location', 'latest', 'count_date', 'peak_hour', 'count_duration', 'adt', 'peaktraffic', 'pccar', 'pclcv', 'pcmcv', 'pchcvi', 'pchcvii', 'pcbus', 'pcheavy', 'NZTMX', 'NZTMY', 'geometry']


In [2]:
traffic.isna().sum().sort_values(ascending=False)

peaktraffic    3251
peak_hour      3217
pcbus          1182
pchcvii        1174
pcheavy        1076
pccar          1076
pclcv          1076
pcmcv          1076
pchcvi         1076
location        298
adt             119
end_name         68
start_name       55
road_name         1
road_id           0
count_date        0
latest            0
geometry          0
dtype: int64

In [3]:
traffic.geometry.is_valid.sum(), len(traffic)
traffic.geometry.notnull().sum()

13093

In [4]:
traffic["adt"].dtype
traffic["adt"].head(10)
traffic["adt"].unique()[:10]  # or .sample(10)
traffic["adt"].isna().sum()

119

In [5]:
traffic = traffic.dropna(subset=["adt", "geometry"])
print("Clean shape:", traffic.shape)

Clean shape: (12974, 18)


In [6]:
#impute missing vehicle category percentages with median
pct_cols = ["pccar", "pclcv", "pcmcv", "pchcvi", "pchcvii", "pcbus", "pcheavy"]
traffic[pct_cols] = traffic[pct_cols].apply(lambda x: x.fillna(x.median()))

#fill missing location names
traffic["location"] = traffic["location"].fillna("unknown")

#drop peak_hour (too many missing)
traffic = traffic.drop(columns=["peak_hour"])

#impute peaktraffic with median
traffic["peaktraffic"] = traffic["peaktraffic"].fillna(traffic["peaktraffic"].median())

#check 'latest' before filtering
if "latest" in traffic.columns:
    print("Latest column unique values:", traffic["latest"].unique())
    #filter if 'Y' exists
    if "Y" in traffic["latest"].unique():
        traffic = traffic[traffic["latest"] == "Y"]

#reset index
traffic.reset_index(drop=True, inplace=True)

#save
processed_folder = r"C:\Users\renhu\OneDrive\Desktop\205.2 DATA\processed"
os.makedirs(processed_folder, exist_ok=True)
output_file = os.path.join(processed_folder, "Average_Daily_Traffic_Clean.geojson")
traffic.to_file(output_file, driver="GeoJSON")

print("Final cleaned dataset saved!")
print("Shape:", traffic.shape)


Latest column unique values: ['Yes']
Final cleaned dataset saved!
Shape: (12974, 17)


In [7]:
#clean up texts 
text_cols = ["road_name", "start_name", "end_name", "location"]
for col in text_cols:
    if col in traffic.columns:
        traffic[col] = traffic[col].astype(str).str.strip().str.lower().replace("nan", np.nan)

In [8]:
num_cols = ["adt", "peaktraffic", "pccar", "pclcv", "pcmcv", "pchcvi", "pchcvii", "pcbus", "pcheavy"]

for col in num_cols:
    if col in traffic.columns:
        #convert to numeric, coerce errors to NaN
        traffic[col] = pd.to_numeric(traffic[col], errors="coerce")

#impute remaining missing with median
traffic[num_cols] = traffic[num_cols].fillna(traffic[num_cols].median())
        

In [19]:
import geopandas as gpd 
import pandas as pd
import numpy as np
import os       

# file path for roadworks
roadworks_fp = r"C:\Users\mathe\Downloads\Roadworks (1).geojson"

# Read the roadworks dataset
roadworks = gpd.read_file(roadworks_fp)
print("Roadworks shape:", roadworks.shape)
print("Roadworks Columns:", list(roadworks.columns))


Roadworks shape: (2519, 15)
Roadworks Columns: ['OBJECTID', 'WorksiteCode', 'WorksiteName', 'ProjectName', 'Status', 'WorksiteType', 'Shape__Area', 'Shape__Length', 'PrincipalOrganisation', 'ProjectStartDate', 'ProjectEndDate', 'WorkStartDate', 'WorkCompletionDate', 'WorkStatus', 'geometry']


In [20]:
# Keeping the relevant columns
roadworks_keep_cols = [
    "WorksiteName",
    "StartDate",
    "EndDate", 
    "Status",
    "WorksiteType",
    "PrincipalOrganisation",
    "WorkStatus",
    "geometry"
]

roadworks = roadworks[[col for col in roadworks_keep_cols if col in roadworks.columns]].copy()
print("Roadworks shape after keeping relevant columns:", roadworks.shape)
print("kept columns:", list(roadworks.columns))

Roadworks shape after keeping relevant columns: (2519, 6)
kept columns: ['WorksiteName', 'Status', 'WorksiteType', 'PrincipalOrganisation', 'WorkStatus', 'geometry']


In [21]:
# Data quality checks
print("\nMissing values in roadworks data:")
print(roadworks.isnull().sum().sort_values(ascending=False))

print(f"\nValid geometries: {roadworks.geometry.is_valid.sum()} out of {len(roadworks)}")


Missing values in roadworks data:
WorksiteName             0
Status                   0
WorksiteType             0
PrincipalOrganisation    0
WorkStatus               0
geometry                 0
dtype: int64

Valid geometries: 2519 out of 2519


In [ ]:
# Cleaning the text columns
text_cols = [
    "WorksiteName",
    "Status",
    "WorksiteType",
    "PrincipalOrganisation",
    "WorkStatus",
    "StartDate",
    "EndDate",
]

for col in text_cols:
    roadworks[col] = roadworks[col].str.strip().str.lower()
    roadworks[col] = roadworks[col].replace("nan", np.nan)

print("\nAfter cleaning text columns:")
print(roadworks[text_cols].head())


After cleaning text columns:
                                        WorksiteName  Status    WorksiteType  \
0           85 custom street east, auckland central.  active  non-excavation   
1  39 hinemoa street, birkenhead - tree maintaine...  active  non-excavation   
2                         ho128- manukau road, epsom  active  non-excavation   
3          55 tauhinu rd, greenhithe  m.0193675.h.25  active      excavation   
4  2406 – 022epa - sky - 20 waimana ave, northcot...  active      excavation   

    PrincipalOrganisation        WorkStatus  
0  shundi customs limited           started  
1        auckland council  pending warranty  
2        auckland council           started  
3          vector limited    ready to start  
4            fuxuan zhang           started  


In [40]:
# Handle date columns
date_cols = ["StartDate", "EndDate"]
for col in date_cols:
	if col in roadworks.columns:
		roadworks[col] = pd.to_datetime(roadworks[col], errors="coerce")
	else:
		print(f"Column '{col}' not found in roadworks DataFrame.")
print("\nAfter converting date columns:")
print(roadworks[[col for col in date_cols if col in roadworks.columns]].dtypes)

Column 'StartDate' not found in roadworks DataFrame.
Column 'EndDate' not found in roadworks DataFrame.

After converting date columns:
Series([], dtype: object)


In [25]:
# Handle missing values and creating new features
categorical_cols = [
    "WorksiteName",
    "Status",
    "WorksiteType",
    "PrincipalOrganisation",
    "WorkStatus",
] 
for col in categorical_cols:
    roadworks[col] = roadworks[col].fillna("unknown")

    if "WorkStatus" in roadworks.columns:
        roadworks["WorkStatus_Simple"] = roadworks["WorkStatus"].apply(
            lambda x: "active" if "start" in str(x) else
            "planned" if "planned" in str(x) else 
             "completed" if "complete" in str(x) else 
             "other"
        )

    roadworks.reset_index(drop=True, inplace=True)
    print("\nAfter handling missing values and creating new features:")
    print(roadworks.head())



After handling missing values and creating new features:
                                        WorksiteName  Status    WorksiteType  \
0           85 custom street east, auckland central.  active  non-excavation   
1  39 hinemoa street, birkenhead - tree maintaine...  active  non-excavation   
2                         ho128- manukau road, epsom  active  non-excavation   
3          55 tauhinu rd, greenhithe  m.0193675.h.25  active      excavation   
4  2406 – 022epa - sky - 20 waimana ave, northcot...  active      excavation   

    PrincipalOrganisation        WorkStatus  \
0  shundi customs limited           started   
1        auckland council  pending warranty   
2        auckland council           started   
3          vector limited    ready to start   
4            fuxuan zhang           started   

                                            geometry WorkStatus_Simple  
0  POLYGON ((174.76882 -36.84511, 174.77029 -36.8...            active  
1  POLYGON ((174.73634 -36.81924

In [26]:
# Saving the cleaned roadworks dataset
processed_folder = r"C:\Users\mathe\Documents\GitHub\205.2-assessment\205.2 DATA\processed"
os.makedirs(processed_folder, exist_ok=True)
output_file = os.path.join(processed_folder, "Roadworks_Clean.geojson")
roadworks.to_file(output_file, driver="GeoJSON")

print("Cleaned roadworks dataset saved!")
print("Shape:", roadworks.shape)
print("Columns:", list(roadworks.columns))

print("\nRoadworks Summary:")
print(roadworks.describe(include='all'))
if "WorksiteType" in roadworks.columns:
    print("\nWorksiteType Value Counts:")
    print(roadworks["WorksiteType"].value_counts())
if "Status" in roadworks.columns:
    print("\nStatus Value Counts:")
    print(roadworks["Status"].value_counts())
if "WorkStatus_Simple" in roadworks.columns:
    print("\nWorkStatus_Simple Value Counts:")
    print(roadworks["WorkStatus_Simple"].value_counts())

Cleaned roadworks dataset saved!
Shape: (2519, 7)
Columns: ['WorksiteName', 'Status', 'WorksiteType', 'PrincipalOrganisation', 'WorkStatus', 'geometry', 'WorkStatus_Simple']

Roadworks Summary:
                                            WorksiteName  Status WorksiteType  \
count                                               2519    2519         2519   
unique                                              2512       2            3   
top     25_151_00-fibre - william pickering dr, rosedale  active   excavation   
freq                                                   2    2516         1829   

       PrincipalOrganisation WorkStatus  \
count                   2519       2519   
unique                   823          6   
top       auckland transport    started   
freq                     450       1560   

                                                 geometry WorkStatus_Simple  
count                                                2519              2519  
unique                      